In [3]:
#Task2 Python functions
#import needed packages
import pyspark
from pyspark import SparkContext #for unabling to set up sc by yourself
from pyspark.sql import SparkSession, SQLContext #spark dataframe = spark sql
from pyspark.sql.functions import to_date, col, count,when,lit,rand,to_timestamp,udf
from pyspark.sql.functions import concat_ws, substring,isnan,regexp_replace,concat,desc
from pyspark.sql.functions import udf,regexp_extract, monotonically_increasing_id,size
from pyspark.ml.feature import Imputer
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType
import pandas as pd
import numpy as np
import os
#building spark session
appname = "courseproject" #define app name
master = "local"

config = pyspark.SparkConf().setAppName(appname)\
.setMaster(master) #we do not have any workers.


#session
#with sql context, create session from it
sc  = SparkContext.getOrCreate(conf=config)
sqlContext = SQLContext(sc)
#don't create many sessions, take up too much room!
sp_session = sqlContext.sparkSession.builder.getOrCreate();
#export data into postgresql
from pyspark import SparkContext, SparkConf, SQLContext


/Library/Spark/spark-3.2.1-bin-hadoop3.2-scala2.13/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
#read in the data from the file

def read_fifa_data(year):
    tablename = "FIFA.PLAYERS_"+ year
    
    players_df = sqlContext.read.format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/postgres")\
    .option("dbtable", tablename)\
    .option("user", "postgres")\
    .option("password", "postgres")\
    .option("Driver", "org.postgresql.Driver")\
    .load()
    players_df.printSchema()
    return players_df
    


In [5]:
#read in all data
df_15 = read_fifa_data("15")
df_16 = read_fifa_data("16")
df_17 = read_fifa_data("17")
df_18 = read_fifa_data("18")
df_19 = read_fifa_data("19")
df_20 = read_fifa_data("20")
df_21 = read_fifa_data("21")
df_22 = read_fifa_data("22")

root
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- DOB: date (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- club_team_id: double (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: integer (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: integer (nullable = true)
 |-- club_joined: string (nullable = true)
 |-- club_contract_valid_until: integer (nullable = true)
 |-- nationality_id: integer (nullable = true)
 |-- nationality_name: string (nullable = true)
 |-- preferred_foot: string (nullable = true)
 |-- weak_foot: integer (nullable = true)
 |-- skill_moves: integer (nullabl

root
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- DOB: date (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- club_team_id: double (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: integer (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: integer (nullable = true)
 |-- club_joined: string (nullable = true)
 |-- club_contract_valid_until: integer (nullable = true)
 |-- nationality_id: integer (nullable = true)
 |-- nationality_name: string (nullable = true)
 |-- preferred_foot: string (nullable = true)
 |-- weak_foot: integer (nullable = true)
 |-- skill_moves: integer (nullabl

root
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- DOB: date (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- club_team_id: double (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: integer (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: integer (nullable = true)
 |-- club_joined: string (nullable = true)
 |-- club_contract_valid_until: integer (nullable = true)
 |-- nationality_id: integer (nullable = true)
 |-- nationality_name: string (nullable = true)
 |-- preferred_foot: string (nullable = true)
 |-- weak_foot: integer (nullable = true)
 |-- skill_moves: integer (nullabl

root
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- DOB: date (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- club_team_id: double (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: integer (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: integer (nullable = true)
 |-- club_joined: string (nullable = true)
 |-- club_contract_valid_until: integer (nullable = true)
 |-- nationality_id: integer (nullable = true)
 |-- nationality_name: string (nullable = true)
 |-- preferred_foot: string (nullable = true)
 |-- weak_foot: integer (nullable = true)
 |-- skill_moves: integer (nullabl

In [1]:
from functools import reduce

def column_add(a,b):
     return  a.__add__(b)
    
def find_skill_sets(df):

    prefix = ["skill","movement","power","mentality","goalkeeping","defending"]
    skill_sets = [] 
    #for each of the skill, find its list of columns
    sum_all = "SumAcrossAllSkillSets"
    for pre in prefix:
        for c in df.columns:
            if c.startswith(pre):
                skill_sets.append(c)
    #add a name to identify players
    player_skill = skill_sets
    player_skill.append("SofifaId")
    df_skill = df.select([c for c in player_skill])\
    .withColumn(sum_all,reduce(column_add,[df[c]  for c in skill_sets]))
    return df_skill
    
def find_players_with_highest_improvement(df1,df2,x):
    print("List of "+ str(x)+" players who achieved highest average improvement across all skill sets")
    df_skillset1 = find_skill_sets(df1)
    df_skillset2 = find_skill_sets(df2).withColumnRenamed("SumAcrossAllSkillSets","New_Sum")\
    .withColumnRenamed("SofifaId","SofifaId2")
    df_avg = df_skillset1.join(df_skillset2,df_skillset1.SofifaId == df_skillset2.SofifaId2)
    df_avg.withColumn("Avg_Improvement",((col("New_Sum") - col("SumAcrossAllSkillSets"))/(len(df_skillset1.columns)-2)).cast("int"))\
    .select("SofifaId","Avg_Improvement").orderBy(desc_nulls_last("Avg_Improvement")).show(x)
    
    
    

In [2]:
#list the players who achieved average highest improvement
find_players_with_highest_improvement(df_15,df_22,10)

NameError: name 'df_15' is not defined

In [39]:
#find y clubs with the most players whose contracts end in 2021
from pyspark.sql.window import Window

windowSpec  = Window.partitionBy("club_name").orderBy("count")
def find_club_with_most_player_due(end_year,df,y):
    df.select("club_name","club_contract_valid_until","club_team_id").filter("club_contract_valid_until == 20"+end_year)\
    .groupBy("club_name","club_team_id").count().orderBy(desc("count")).show(y)
    
    
    

In [39]:
find_club_with_most_player_due("21","15",10)
find_club_with_most_player_due("21","21",10)



for year 15
+--------------------+------------+-----+
|           club_name|club_team_id|count|
+--------------------+------------+-----+
|      Uniautónoma FC|    112830.0|   18|
|     Boyacá Chicó FC|    112019.0|   17|
|  Defensa y Justicia|    111710.0|   15|
|       Santos Laguna|    110144.0|   13|
|Club Atlético Ban...|    110404.0|   13|
|       CD Huachipato|    111327.0|   13|
|US Orléans Loiret...|    112276.0|   13|
| Atlético de Rafaela|    110392.0|   12|
|         Club Olimpo|    110398.0|   12|
|     Rosario Central|    110580.0|   12|
+--------------------+------------+-----+
only showing top 10 rows

for year 21
+--------------------+------------+-----+
|           club_name|club_team_id|count|
+--------------------+------------+-----+
|         Yokohama FC|    113197.0|   30|
|      Kashiwa Reysol|    101145.0|   29|
|        Oita Trinita|    113183.0|   28|
|        Cerezo Osaka|    101148.0|   28|
|      Vegalta Sendai|    112836.0|   26|
|         Exeter City|    

In [83]:
#find  z clubs with the largest number of players in the data set where z >=5
def find_clubs_with_most_players(z,df):
    print("for this year:")
    club_player_count = df.select("club_name","club_team_id").filter("club_name != 'null'")\
    .groupBy("club_name").count().orderBy(desc_nulls_last("count")).limit(z)
    if club_player_count.select("count").distinct().count() ==1:
        print(" all teams have the same amount of players")
    club_player_count.show()
    

In [84]:
find_clubs_with_most_players(20,df_15)
find_clubs_with_most_players(20,df_16)
find_clubs_with_most_players(20,df_17)
find_clubs_with_most_players(20,df_22)




for this year:
 all teams have the same amount of players
+--------------------+-----+
|           club_name|count|
+--------------------+-----+
|         Hannover 96|   33|
|                Roma|   33|
|   Manchester United|   33|
|           SC Bastia|   33|
|             Arsenal|   33|
|       FC Schalke 04|   33|
|          Sunderland|   33|
|        Swansea City|   33|
|               Lazio|   33|
|          Stoke City|   33|
|RCD Espanyol de B...|   33|
|  Olympique Lyonnais|   33|
|    Newcastle United|   33|
|Deportivo de La C...|   33|
|      Crystal Palace|   33|
|             Burnley|   33|
|         Aston Villa|   33|
|     Manchester City|   33|
| Queens Park Rangers|   33|
|               Genoa|   33|
+--------------------+-----+

for this year:
+--------------------+-----+
|           club_name|count|
+--------------------+-----+
|          Sunderland|   32|
|             Arsenal|   32|
|        ESTAC Troyes|   32|
|         Hannover 96|   32|
|    AS Saint-Étienne|   32

In [95]:
#find most frequent nation position and team position
def find_most_freq_np_tp(df):
    #for the team position
    if "team_position" in df.columns:
        print("Most frequen team position is:")
        df.select("team_position").groupBy("team_position").count().orderBy(desc("count")).show(1)
    else:
        print("Most frequen team position is:")
        df.select("club_position").groupBy("club_position").count().orderBy(desc("count")).show(1)
    #for the nation position
    if "nation_position" in df.columns:
        print("Most frequen nation position is:")
        df.select("nation_position").groupBy("nation_position").count().orderBy(desc("count")).show(1)
    else:
        print("There is no enough data about nation position in this dataset")

In [96]:
find_most_freq_np_tp(df_15)
find_most_freq_np_tp(df_16)
find_most_freq_np_tp(df_17)
find_most_freq_np_tp(df_18)
find_most_freq_np_tp(df_19)
find_most_freq_np_tp(df_20)
find_most_freq_np_tp(df_21)
find_most_freq_np_tp(df_22)

Most frequen team position is:
+-------------+-----+
|club_position|count|
+-------------+-----+
|          SUB| 6906|
+-------------+-----+
only showing top 1 row

There is no enough data about nation position in this dataset
Most frequen team position is:
+-------------+-----+
|club_position|count|
+-------------+-----+
|          SUB| 6724|
+-------------+-----+
only showing top 1 row

There is no enough data about nation position in this dataset
Most frequen team position is:
+-------------+-----+
|club_position|count|
+-------------+-----+
|          SUB| 7502|
+-------------+-----+
only showing top 1 row

There is no enough data about nation position in this dataset
Most frequen team position is:
+-------------+-----+
|club_position|count|
+-------------+-----+
|          SUB| 7636|
+-------------+-----+
only showing top 1 row

There is no enough data about nation position in this dataset
Most frequen team position is:
+-------------+-----+
|club_position|count|
+-------------+--

In [85]:
#find the most popular nationality of players
def find_most_popular_nationality(df):
    print("most famous nationality for this year is: ")
    df.select("nationality_id","nationality_name").groupBy("nationality_id","nationality_name").count().orderBy(desc("count")).show(1)

In [86]:
find_most_popular_nationality(df_15)
find_most_popular_nationality(df_22)

most famous nationality for this year is: 
+--------------+----------------+-----+
|nationality_id|nationality_name|count|
+--------------+----------------+-----+
|            14|         England| 1627|
+--------------+----------------+-----+
only showing top 1 row

most famous nationality for this year is: 
+--------------+----------------+-----+
|nationality_id|nationality_name|count|
+--------------+----------------+-----+
|            14|         England| 1719|
+--------------+----------------+-----+
only showing top 1 row

